In [111]:
import datasets
import helpers
import pickle
import numpy





In [112]:
results = helpers.load_all_results()

In [113]:
print(len(results))

3


In [124]:
result = results[0]

In [125]:
result.keys()

dict_keys(['name', 'res', 'dataset', 'dist_threshold'])

In [126]:
def get_true_prob(result):
    with open(datasets.get_dataset_fn(result['dataset']), 'rb') as f:
        data, queries, groundtruth = pickle.load(f)

    A = numpy.array([len(x) for x in groundtruth])
    return 1/A

In [127]:
def get_result_prob(results):
    probs = {}
    for q, res in results.items():
        probs[q] = []
        cnts = {}
        for r in res:
            if r != -1:
                cnts.setdefault(r, 0)
                cnts[r] += 1
        n = sum(cnts.values())
        p = [cnts[r]/n for r in cnts]
        probs[q] = p
    return probs



In [128]:
def total_variation_dist(g, res):
    all = numpy.zeros(len(g))
    for i in range(len(g)):
        n = int(1/g[i])
        r = numpy.array(res[i] + [0] * (n - len(res[i])))
        all[i] = 0.5 * numpy.sum(numpy.abs(r - g[i]))
    return numpy.mean(all)

def total_variation_dist_only_result(res):
    all = numpy.zeros(len(res))
    for i in range(len(res)):
        n = len(res[i])
        if n == 0:
            continue
        r = numpy.array(res[i] + [0] * (n - len(res[i])))
        all[i] = 0.5 * numpy.sum(numpy.abs(r - 1/n))
    return numpy.mean(all)   




In [129]:
p = get_result_prob(result['res'])

print(total_variation_dist(get_true_prob(result), p))
print(total_variation_dist_only_result(p))



0.6929097104079931
0.03573368613021034
